In [69]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [ ]:
# import MNIST DATASET and DataLoader

train_dataset = torchvision.datasets.MNIST(root='./data',train=True, transform=transforms.ToTensor(), download=True)

test_dataset = torchvision.datasets.MNIST(root='./data',train=False, transform=transforms.ToTensor(),download = True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=100, shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,  batch_size=100, shuffle=False)

In [115]:
# Summary :
        
# Calculation
# [nChannel @ heightxwidth]
# input 28x28
# conv2d 32@ 28x28 (28 - 5 + 4)/1 +1
# relu
# max pool 2d 32@ 14x14 (28 - 2 + 0)/2 + 1
# conv2d 64@ 14x14 (14 - 5 + 4)/1 + 1
# relu
# max pool 2d 64@ 7x7 (14 - 2 + 0)/2 + 1
# Flatten 64*7*7 nChannel*height*width
# 64*7*7 -> 248
# relu
# 248 -> 100
# relu
# 100 -> 10

class Net(nn.Module):
    
    def __init__(self):
        
        super(Net, self).__init__()
        # 1. input 2. output 3. window 4. stride 5. pad
        self.conv1 = nn.Conv2d(1,32,5,1,2)
        self.conv2 = nn.Conv2d(32,64,5,1,2)
        # affine operation 1. input 2.output
        self.fc1 = nn.Linear(7*7*64, 248)
        self.fc2 = nn.Linear(248, 100)
        self.fc3 = nn.Linear(100, 10)
        
    def forward(self, x):
        # 2D conv -> relu -> max-pooling 2 window 2 stride
        x = F.max_pool2d(F.relu(self.conv1(x)),2,2)
        x = F.max_pool2d(F.relu(self.conv2(x)),2,2)
        # Flatten (7*7*64)
        x = x.reshape(x.size(0),-1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

In [117]:
model = Net()

# Loss : Cross Entropy and optimizer : Stochastic Gradient Descent
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [118]:
# Train the model
total_step = len(train_loader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Run the forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        # perform SGD optimization and Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

Epoch [1/5], Step [100/600], Loss: 2.3008, Accuracy: 8.00%
Epoch [1/5], Step [200/600], Loss: 2.2851, Accuracy: 10.00%
Epoch [1/5], Step [300/600], Loss: 2.2798, Accuracy: 17.00%
Epoch [1/5], Step [400/600], Loss: 2.2497, Accuracy: 41.00%
Epoch [1/5], Step [500/600], Loss: 2.1839, Accuracy: 56.00%
Epoch [1/5], Step [600/600], Loss: 1.9242, Accuracy: 74.00%
Epoch [2/5], Step [100/600], Loss: 1.1196, Accuracy: 69.00%
Epoch [2/5], Step [200/600], Loss: 0.7119, Accuracy: 76.00%
Epoch [2/5], Step [300/600], Loss: 0.5874, Accuracy: 83.00%
Epoch [2/5], Step [400/600], Loss: 0.4609, Accuracy: 84.00%
Epoch [2/5], Step [500/600], Loss: 0.4379, Accuracy: 82.00%
Epoch [2/5], Step [600/600], Loss: 0.2791, Accuracy: 90.00%
Epoch [3/5], Step [100/600], Loss: 0.3898, Accuracy: 88.00%
Epoch [3/5], Step [200/600], Loss: 0.2996, Accuracy: 92.00%
Epoch [3/5], Step [300/600], Loss: 0.3358, Accuracy: 91.00%
Epoch [3/5], Step [400/600], Loss: 0.2147, Accuracy: 93.00%
Epoch [3/5], Step [500/600], Loss: 0.3066

In [119]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format((correct / total) * 100))

Test Accuracy of the model on the 10000 test images: 95.34 %


Change Optimizer SGD -> ADAM

In [120]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [121]:
# Train the model
total_step = len(train_loader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Run the forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        # perform SGD optimization and Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

Epoch [1/5], Step [100/600], Loss: 0.2418, Accuracy: 92.00%
Epoch [1/5], Step [200/600], Loss: 0.2561, Accuracy: 93.00%
Epoch [1/5], Step [300/600], Loss: 0.1035, Accuracy: 94.00%
Epoch [1/5], Step [400/600], Loss: 0.1294, Accuracy: 95.00%
Epoch [1/5], Step [500/600], Loss: 0.0692, Accuracy: 96.00%
Epoch [1/5], Step [600/600], Loss: 0.0443, Accuracy: 99.00%
Epoch [2/5], Step [100/600], Loss: 0.0592, Accuracy: 98.00%
Epoch [2/5], Step [200/600], Loss: 0.0993, Accuracy: 95.00%
Epoch [2/5], Step [300/600], Loss: 0.1872, Accuracy: 96.00%
Epoch [2/5], Step [400/600], Loss: 0.1860, Accuracy: 94.00%
Epoch [2/5], Step [500/600], Loss: 0.1069, Accuracy: 96.00%
Epoch [2/5], Step [600/600], Loss: 0.0870, Accuracy: 97.00%
Epoch [3/5], Step [100/600], Loss: 0.0747, Accuracy: 97.00%
Epoch [3/5], Step [200/600], Loss: 0.0377, Accuracy: 98.00%
Epoch [3/5], Step [300/600], Loss: 0.0994, Accuracy: 97.00%
Epoch [3/5], Step [400/600], Loss: 0.0611, Accuracy: 97.00%
Epoch [3/5], Step [500/600], Loss: 0.077

In [122]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format((correct / total) * 100))

Test Accuracy of the model on the 10000 test images: 97.08 %


Reference : https://adventuresinmachinelearning.com/convolutional-neural-networks-tutorial-in-pytorch/